Name: Ziwei Liu
Student ID: 3766789

In [1]:
import os
import zipfile
import torch

import requests
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import STL10
from tqdm import tqdm
import torch.nn as nn

In [2]:
from examples.models.densenet import densenet121

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [3]:
threads = 16
torch.set_num_threads(threads)

#maybe better performance (Jupyter)
#%env OMP_PLACES=cores
#%env OMP_PROC_BIND=close
#%env OMP_WAIT_POLICY=active

#WSL
os.environ['OMP_PLACES'] = 'cores'
os.environ['OMP_PROC_BIND'] = 'close'
os.environ['OMP_WAIT_POLICY'] = 'active'

In [4]:
#axx_mult = 'mul8s_acc'


In [5]:
axx_mult = 'exact'
model = densenet121(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_exact/build.ninja...
Building extension module PyInit_conv2d_exact...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading exten

DenseNet(
  (features): Sequential(
    (conv0): AdaPT_Conv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
      (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
    )
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): AdaPT_Conv2d(
          64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False
          (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
          (quantizer_w): TensorQuant

In [ ]:
import torch
from torchvision import transforms as T
from torchvision.datasets import STL10
from torch.utils.data import DataLoader


def val_dataloader(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225)):

    transform = T.Compose(
        [
            T.Resize(224),  # Resize to 224x224 before applying normalization
            T.ToTensor(),
            T.Normalize(mean, std),
        ]
    )
    dataset = STL10(root="datasets/stl10_data", split='test', download=True, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=1024,
        num_workers=0,
        drop_last=True,
        pin_memory=False,
    )
    return dataloader


transform = T.Compose(
    [
        T.Resize(224),  # Resize to 224x224
        T.RandomCrop(224),  # Apply random crop to 224x224
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225)),
    ]
)


dataset = STL10(root="datasets/stl10_data", split='train', download=True, transform=transform)


evens = list(range(0, len(dataset), 10))
trainset_1 = torch.utils.data.Subset(dataset, evens)


data = val_dataloader()


data_t = DataLoader(trainset_1, batch_size=128, shuffle=False, num_workers=0)


data_iter = iter(data)
images, labels = next(data_iter)

print(images.shape)
print(labels.shape)


Files already downloaded and verified
Files already downloaded and verified
torch.Size([1024, 3, 224, 224])
torch.Size([1024])


In [7]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib

def collect_stats(model, data_loader, num_batches):
     """Feed data to the network and collect statistic"""

     # Enable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.disable_quant()
                 module.enable_calib()
             else:
                 module.disable()

     for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
         model(image.cpu())
         if i >= num_batches:
             break

     # Disable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.enable_quant()
                 module.disable_calib()
             else:
                 module.enable()

def compute_amax(model, **kwargs):
 # Load calib result
 for name, module in model.named_modules():
     if isinstance(module, quant_nn.TensorQuantizer):
         if module._calibrator is not None:
             if isinstance(module._calibrator, calib.MaxCalibrator):
                 module.load_calib_amax()
             else:
                 module.load_calib_amax(**kwargs)
         print(F"{name:40}: {module}")
 model.cpu()

# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)

100%|██████████| 2/2 [04:59<00:00, 149.59s/it]
W0222 20:39:57.184715 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.185098 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.185382 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.185596 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.185798 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.185988 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.186201 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.186431 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.186669 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.186898 140123427950784 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 20:39:57.187138 140123427950784 tenso

features.conv0.quantizer                : TensorQuantizer(8bit per-tensor amax=2.6387 calibrator=HistogramCalibrator quant)
features.conv0.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.1544 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.2427 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0649 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer: TensorQuantizer(8bit per-tensor amax=0.0687 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0350 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.5290 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax

In [ ]:
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

 57%|█████▋    | 4/7 [38:12<28:42, 574.01s/it]

In [ ]:
axx_mult = 'appro1'
model = densenet121(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_approx1/build.ninja...
Building extension module PyInit_conv2d_approx1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/2] c++ -MMD -MF axx_conv2d.o.d -DTORCH_EXTENSION_NAME=PyInit_conv2d_approx1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /root/miniconda3/lib/python3.8/site-packages/torch/include -isystem /root/miniconda3/lib/python3.8/site-packages/torch/include/torch/csrc/api/include -isystem /root/miniconda3/lib/python3.8/site-packages/torch/include/TH -isystem /root/miniconda3/lib/python3.8/site-packages/torch/include/THC -isystem /root/miniconda3/include/python3.8 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++14 -DAXX_MULT=approx1 -march=native -fopenmp -O3 -c /root/autodl-tmp/adapt

RuntimeError: Error building extension 'PyInit_conv2d_approx1'

In [ ]:
axx_mult = 'appro2'
model = DensNet121(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Epoch 1/15
Epoch: [0]  [ 0/40]  eta: 0:11:05  lr: 0.0001  img/s: 7.704564943027618  loss: 1.1255 (1.1255)  acc1: 63.2812 (63.2812)  acc5: 88.2812 (88.2812)  time: 16.6266  data: 0.0131
Epoch: [0]  [ 1/40]  eta: 0:10:11  lr: 0.0001  img/s: 8.70813929275048  loss: 0.8878 (1.0066)  acc1: 63.2812 (67.5781)  acc5: 88.2812 (92.9688)  time: 15.6701  data: 0.0139
Epoch: [0]  [ 2/40]  eta: 0:10:30  lr: 0.0001  img/s: 6.96266787160591  loss: 0.9254 (0.9796)  acc1: 70.3125 (68.4896)  acc5: 96.8750 (94.2708)  time: 16.5797  data: 0.0143
Epoch: [0]  [ 3/40]  eta: 0:09:47  lr: 0.0001  img/s: 9.294946305056063  loss: 0.9140 (0.9632)  acc1: 70.3125 (68.9453)  acc5: 94.5312 (94.3359)  time: 15.8813  data: 0.0145
Epoch: [0]  [ 4/40]  eta: 0:09:16  lr: 0.0001  img/s: 9.295206367358771  loss: 0.9140 (0.9414)  acc1: 70.3125 (70.1562)  acc5: 94.5312 (93.9062)  time: 15.4620  data: 0.0145
Epoch: [0]  [ 5/40]  eta: 0:09:10  lr: 0.0001  img/s: 7.4865759633279865  loss: 0.9140 (0.9376)  acc1: 70.3125 (70.1823) 

In [ ]:
axx_mult = 'appro3'
model = DensNet121(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))